In [ ]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, TextStreamer, AutoTokenizer,AutoModelForVision2Seq
from PIL import Image
import requests
import torch
from datasets import load_dataset
from huggingface_hub import login
from tqdm import tqdm
import os
import google.generativeai as genai
import time
import re

In [ ]:
login("YOUR_API_KEY")

In [ ]:
ds = load_dataset("Ayush-Singh/llms-are-blind-captions_florence2-large")
ds

In [ ]:
model_id = "BUAADreamer/PaliGemma-3B-Chat-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForVision2Seq.from_pretrained(model_id, device_map="cuda")

In [ ]:
def get_accuracy(l1,l2):
    count = 0
    for x,y in zip(l1,l2):
        if x==y:
            count+=1
    return count/len(l1)*100

def generate(model,processor,tokenizer,dataset,prompt=None):
    results = []
    for sample in tqdm(dataset):
        image = sample['image']
        if prompt==None:
            prompt = sample['prompt']
        length = len(prompt)
        #print(prompt)
        pixel_values = processor(images=image, return_tensors="pt").to(model.device)["pixel_values"]

        messages = prompt
        
        input_ids = tokenizer.encode(messages, return_tensors="pt")
        image_token_id = tokenizer.convert_tokens_to_ids("<image>")
        image_prefix = torch.empty((1, getattr(processor, "image_seq_length")), dtype=input_ids.dtype).fill_(image_token_id)
        input_ids = torch.cat((image_prefix, input_ids), dim=-1).to(model.device)

        generation = model.generate(input_ids, pixel_values=pixel_values, max_new_tokens=10)
        generation = generation[0]
        decoded = processor.decode(generation, skip_special_tokens=True)[length:]
        #print(decoded)
        results.append(decoded)
        
    return results

### Geometry - Circled Letters

In [ ]:
Sample_Circled_Letter = ds['Circled_Letter'].select(range(200))
Sample_Circled_Letter

In [ ]:
result_cl = generate(model,processor,tokenizer,Sample_Circled_Letter)

In [ ]:
print(result_cl)

In [ ]:
result_cl = [item.strip() for item in result_cl]

In [ ]:
print(result_cl)

In [ ]:
count = 0
for x,y in zip(result_cl,Sample_Circled_Letter['groundtruth']):
    if (x==y):
        count+=1
print("Accuracy is = ", count/len(result_cl)*100)

### **Geometry** - Touching Circles

In [ ]:
Sample_Touching_Circles = ds['Touching_Circles'].select(range(200))
Sample_Touching_Circles

In [ ]:
result_tc = generate(model,processor,tokenizer,Sample_Touching_Circles)

In [ ]:
print(result_tc)

In [ ]:
count = 0
for x,y in zip(result_tc,Sample_Touching_Circles['groundtruth']):
    if (x=='yes' and y=='Yes') or (x=='no' and y=='No'):
        count+=1
print("Accuracy is = ", count/len(result_tc)*100)

### **Geometry** - Are Line Intersecting?

In [ ]:
Sample_Line_Intersecting = ds['Line_Plot_Intersections'].select(range(200))
Sample_Line_Intersecting

In [ ]:
prompt = "Are the two line plots intersecting? Answer with Yes/No. "
result_lp = generate(model,processor,tokenizer,Sample_Line_Intersecting,prompt)

In [ ]:
print(result_lp)

In [ ]:
count = 0
for x,y in zip(result_lp,Sample_Line_Intersecting['groundtruth']):
    if (x=="yes" and y!=0 or x=="no" and y==0):
        count+=1
print("Accuracy is = ", count/len(result_lp)*100)

### **Counting** - Olympic Rings

In [ ]:
Sample_Olympic_Ring = ds['Olympic_Counting_Circles'].select(range(200))
Sample_Olympic_Ring

In [ ]:
prompt = "Count the total number of circles in the image? "
result_or = generate(model,processor,tokenizer,Sample_Olympic_Ring)

In [ ]:
print(result_or)

In [ ]:
print("Accuracy is = ", get_accuracy(result_or,Sample_Olympic_Ring['groundtruth']))

### **Counting** - Number of Line Intersections

In [ ]:
Sample_Line_Point_Int = ds['Line_Plot_Intersections'].select(range(200))
Sample_Line_Point_Int

In [ ]:
prompt = "Count the number of points of intersection of red and blue lines?"
result_lpi = generate(model,processor,tokenizer,Sample_Line_Point_Int,prompt)

In [ ]:
print(result_lpi)

In [ ]:
print(Sample_Line_Point_Int['groundtruth'])

In [ ]:
print("Accuracy is = ", get_accuracy(result_lpi,Sample_Line_Point_Int['groundtruth']))

### **Counting** - RowsxColumns

In [ ]:
Sample_Rows = ds['Counting_Grid_Blank_Grids'].select(range(200))
Sample_Rows

In [ ]:
prompt = "Count number of rows and number columns in grid "
result_cr = generate(model,processor,tokenizer,Sample_Rows,prompt)

In [ ]:
print(result_cr[:20])

In [ ]:
def extract_numbers(s):
    numbers = re.findall(r'\d+', s)
    return f'{numbers[0]},{numbers[1]}' if len(numbers) >= 2 else None

extracted_numbers = [extract_numbers(s) for s in result_cr if extract_numbers(s)]

In [ ]:
extracted_numbers[:10]

In [ ]:
print("Accuracy is = ", get_accuracy(extracted_numbers,Sample_Rows['groundtruth']))

### **Counting** - Subway Connections

In [ ]:
Sample_Subway = ds['Subway_Connections'].select(range(10))
Sample_Subway

In [ ]:
result_sc = generate(model,processor,tokenizer,Sample_Subway)

In [ ]:
print(result_sc)

### **Counting** - Nested Squares

In [ ]:
Sample_Nested = ds['Nested_Squares'].select(range(200))
Sample_Nested

In [ ]:
prompt = "Count total number of squares in the image."
result_ns = generate(model,processor,tokenizer,Sample_Nested,prompt)

In [ ]:
print(result_ns)

In [ ]:
print("Accuracy is = ",get_accuracy(result_ns, Sample_Nested['groundtruth']))